In [2]:
!pip install torch torchvision

In [3]:
#Transforming dataset from absolute values in relative values

In [42]:
import pandas as pd

image_width = 676
image_height = 380

matrix = pd.read_csv("data.csv")

In [43]:
matrix = pd.read_csv("data.csv")

matrix['xcenter'] = matrix['xcenter'] / image_width
matrix['ycenter'] = matrix['ycenter'] / image_height
matrix['width'] = matrix['width'] / image_width
matrix['heigth'] = matrix['heigth'] / image_height

df = pd.DataFrame(matrix)

df.to_csv('relative_dimentions_dataset.csv', index=False)

In [44]:
#creating train.csv file that has 2 columns: "image" -> name of the image, "label" -> name of the label txt file

In [45]:
dt_train_matrix = df
dt_train_matrix = dt_train_matrix.drop(columns=["xcenter"])
dt_train_matrix = dt_train_matrix.drop(columns=["ycenter"])
dt_train_matrix = dt_train_matrix.drop(columns=["width"])
dt_train_matrix = dt_train_matrix.drop(columns=["heigth"])
dt_train_matrix['txt'] = dt_train_matrix['image']
dt_train_matrix['txt'] = dt_train_matrix['txt'].astype(str).replace("jpg", "txt", regex=True)
dt_train_matrix.to_csv('train.csv', index=False, header=False)

In [46]:
#creating all label files and writing inside the values of the label (class, xcenter, ycenter, width, heigth)

In [50]:
for index, row in df.iterrows():
    with open('labels/'+ row['image'].replace(".jpg", "") +'.txt', 'w') as file:
        file.write('0 ' + str(row['xcenter']) + ' ' + str(row['ycenter']) + ' '  + str(row['width']) + ' '  + str(row['heigth']))

In [24]:
"""
Creates a Pytorch dataset to load the Pascal VOC dataset
"""

import torch
import os
import pandas as pd
from PIL import Image


class VOCDataset(torch.utils.data.Dataset):
    def __init__(
        self, csv_file, img_dir, label_dir, S=7, B=2, C=1, transform=None,
    ):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.S = S #split size
        self.B = B #number of bounding boxes
        self.C = C

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
        boxes = []
        with open(label_path) as f:
            for label in f.readlines():
                class_label, x, y, width, height = [
                    float(x) if float(x) != int(float(x)) else int(x)
                    for x in label.replace("\n", "").split()
                ]

                boxes.append([class_label, x, y, width, height])

        img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path)
        boxes = torch.tensor(boxes)

        if self.transform:
            # image = self.transform(image)
            image, boxes = self.transform(image, boxes)

        # Convert To Cells
        label_matrix = torch.zeros((self.S, self.S, self.C + 5 * self.B))

        for box in boxes:
            class_label, x, y, width, height = box.tolist()
            class_label = int(class_label)

            # i,j represents the cell row and cell column
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i

            """
            Calculating the width and height of cell of bounding box,
            relative to the cell is done by the following, with
            width as the example:
            
            width_pixels = (width*self.image_width)
            cell_pixels = (self.image_width)
            
            Then to find the width relative to the cell is simply:
            width_pixels/cell_pixels, simplification leads to the
            formulas below.
            """
            width_cell, height_cell = (
                width * self.S,
                height * self.S,
            )

            # If no object already found for specific cell i,j
            # Note: This means we restrict to ONE object
            # per cell!
            if label_matrix[i, j, 1] == 0:
                # Set that there exists an object
                label_matrix[i, j, 1] = 1

                # Box coordinates
                box_coordinates = torch.tensor(
                    [x_cell, y_cell, width_cell, height_cell]
                )

                label_matrix[i, j, 2:6] = box_coordinates

                # Set one hot encoding for class_label
                label_matrix[i, j, class_label] = 1

        return image, label_matrix

In [23]:
test_dataset = VOCDataset("train.csv", img_dir="images/", label_dir="labels/",)
test_dataset[2]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=676x380>,
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [1.0000, 1.0000, 0.6664, 0.0629, 0.3951, 0.3115, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000]]]))